In [1]:
## Import Ticker class from yahooquery library 
# To access data from yahooquery, an argument (Company ticker symbol) is passed to the Ticker class 

from yahooquery import Ticker
import pandas as pd

In [2]:
## Create instance of Ticker class (Company ticker symbol)

WES_ASX = Ticker(['WES.AX'])

In [3]:
## Market Value Ratios 

# Market value ratios are used to evaluate the current share price of a company 
# Typically the objective of market value ratios is to identify undervalued or overvalued companies

# The ratio formulas comprise of data from the company's financial statements, as well as technical elements from the stock performance

In [4]:
# Select the Income Statement from the Ticker class, this will return a DataFrame

incomeStatement = WES_ASX.income_statement().reset_index()

incomeStatement = incomeStatement.loc[incomeStatement['periodType'] == '12M']

In [5]:
# Select History from Ticker class, this will return a DataFrame of historical stock price data 
# A period of 5 years has been selected for historical analysis 

stockPrice = WES_ASX.history(period="5y")

# Print results 

stockPrice = stockPrice.reset_index()

In [6]:
# Convert the date object to datetime, as this will be required when merging the two DataFrames  

stockPrice['date'] = pd.to_datetime(stockPrice['date'])

In [7]:
# Price to Earnings Ratio (P/E)

# Extract each required column for P/E Ratio calculation from the Income Statement object 

# Reporting Period

reportingPeriod = incomeStatement['asOfDate']

# Basic Earnings per Share (EPS)

Basic_EPS = incomeStatement['BasicEPS']

# Diluted Earnings per Share (EPS)

Diluted_EPS = incomeStatement['DilutedEPS']

# The differentiating factor between Basic EPS & Diluted EPS is the former accounts for shares outstanding only
# Whereas the latter accounts for shares outstanding + all additional potential dilution (e.g. stock options, convertible securities)


## Create new DataFrame containing each required column with pd.concat method, which concentrates DataFrame objects
# Add relevant keys for each column 

PE_Analysis = pd.concat([reportingPeriod, Basic_EPS, Diluted_EPS], 
                         axis=1, keys=['date', 'Basic EPS', 'Diluted EPS'])


# Print results

PE_Analysis

,date,Basic EPS,Diluted EPS
0,2019-06-30,4.8718,4.8675
1,2020-06-30,1.5004,1.4991
3,2021-06-30,2.1040,2.1020
4,2022-06-30,2.0780,2.0760


In [8]:
## Create merged DataFrame, using the pd.merge method  
# 'How' argument refers to how the obects will be merged 
# An inner join will return reocrds with matching values only on the specified column 
# In this instance, the objects will be joined on the date column 

PE_Analysis = pd.merge(stockPrice, PE_Analysis, how='inner', on='date')

In [9]:
# Print results 

PE_Analysis

,symbol,date,open,high,low,close,volume,adjclose,dividends,Basic EPS,Diluted EPS
0,WES.AX,2020-06-30,44.889999,45.310001,44.439999,44.830002,3359648,39.605801,0.0,1.5004,1.4991
1,WES.AX,2021-06-30,58.900002,59.599998,58.580002,59.099998,1526860,53.294762,0.0,2.1040,2.1020
2,WES.AX,2022-06-30,42.500000,42.689999,41.910000,41.910000,3137351,40.324638,0.0,2.0780,2.0760


In [10]:
# Calculate PE Ratio using the stock price at the date in which the financial results were released 
# The PE Ratio is calculated by diving the closing price by the EPS 

PE_Analysis['PE Ratio (Basic)'] = PE_Analysis['close'] / PE_Analysis['Basic EPS'] 

# Print results

PE_Analysis

,symbol,date,open,high,low,close,volume,adjclose,dividends,Basic EPS,Diluted EPS,PE Ratio (Basic)
0,WES.AX,2020-06-30,44.889999,45.310001,44.439999,44.830002,3359648,39.605801,0.0,1.5004,1.4991,29.878700
1,WES.AX,2021-06-30,58.900002,59.599998,58.580002,59.099998,1526860,53.294762,0.0,2.1040,2.1020,28.089353
2,WES.AX,2022-06-30,42.500000,42.689999,41.910000,41.910000,3137351,40.324638,0.0,2.0780,2.0760,20.168431


In [11]:
## Price to Book Ratio 

# Select the Balance Sheet from the Ticker class, this will return a DataFrame

balanceSheet = WES_ASX.balance_sheet().reset_index()

balanceSheet

,symbol,asOfDate,periodType,currencyCode,AccountsPayable,AccountsReceivable,AccumulatedDepreciation,CapitalLeaseObligations,CapitalStock,CashAndCashEquivalents,...,StockholdersEquity,TangibleBookValue,TotalAssets,TotalCapitalization,TotalDebt,TotalEquityGrossMinorityInterest,TotalLiabilitiesNetMinorityInterest,TotalNonCurrentAssets,TotalNonCurrentLiabilitiesNetMinorityInterest,WorkingCapital
0,WES.AX,2019-06-30,12M,AUD,3.620000e+09,1.027000e+09,-4.587000e+09,NaN,1.572800e+10,7.950000e+08,...,9.971000e+09,5.895000e+09,1.833300e+10,1.264400e+10,3.029000e+09,9.971000e+09,8.362000e+09,1.198300e+10,3.146000e+09,1.134000e+09
1,WES.AX,2020-06-30,12M,AUD,4.008000e+09,8.900000e+08,-6.272000e+09,7.242000e+09,1.581800e+10,2.913000e+09,...,9.344000e+09,4.717000e+09,2.542500e+10,1.149700e+10,9.898000e+09,9.344000e+09,1.608100e+10,1.736100e+10,8.811000e+09,7.940000e+08
2,WES.AX,2021-06-30,12M,AUD,4.234000e+09,1.070000e+09,-7.221000e+09,7.105000e+09,1.582600e+10,3.023000e+09,...,9.715000e+09,5.812000e+09,2.621400e+10,1.178700e+10,1.012700e+10,9.715000e+09,1.649900e+10,1.711800e+10,8.584000e+09,1.181000e+09
3,WES.AX,2022-06-30,12M,AUD,5.362000e+09,1.938000e+09,-8.430000e+09,7.123000e+09,1.357400e+10,7.050000e+08,...,7.981000e+09,3.330000e+09,2.727100e+10,1.195100e+10,1.208100e+10,7.981000e+09,1.929000e+10,1.766600e+10,1.039700e+10,7.120000e+08


In [12]:
# Extract each required column for Price to Book Ratio calculation from the Balance Sheet object 

# Shareholder Equity

shareholderEquity = balanceSheet['StockholdersEquity']

# Shares Outstanding 

sharesOutstanding = balanceSheet['ShareIssued']

# 


PB_Analysis = pd.concat([reportingPeriod, shareholderEquity, sharesOutstanding], axis=1,
                       keys=['Reporting Period', 'Shareholder Equity', 'Shares Outstanding']).dropna()

In [13]:
PB_Analysis

,Reporting Period,Shareholder Equity,Shares Outstanding
0,2019-06-30,9.971000e+09,1.133840e+09
1,2020-06-30,9.344000e+09,1.133840e+09
3,2021-06-30,7.981000e+09,1.134145e+09
